<a href="https://colab.research.google.com/github/doartel/test/blob/master/Test_connect_Alphavantage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installe les bibliothèques nécessaires si ce n'est pas fait

In [1]:
!pip install transformers

Indique ta clé API Alpha Vantage

In [2]:
ALPHA_VANTAGE_API_KEY = "QOBLTNNLNAPAK60L"

Code pour afficher l'analyse de sentiment

In [5]:
import requests
import pandas as pd
from transformers import pipeline

# Configuration
ALPHA_VANTAGE_URL = "https://www.alphavantage.co/query"
TICKER = "AAPL"
LIMIT = 10

# Fonction pour récupérer des news financières
def get_financial_news(api_key, ticker="AAPL", limit=10):
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "apikey": api_key,
        "limit": limit
    }
    response = requests.get(ALPHA_VANTAGE_URL, params=params)

    if response.status_code == 200:
        news_data = response.json().get("feed", [])
        return news_data[:limit]
    else:
        print("Erreur lors de la requête:", response.text)
        return []

# Fonction pour analyser les sentiments
def analyze_sentiment_with_finbert(news_articles):
    # Charger le pipeline FinBERT
    sentiment_pipeline = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone")

    sentiment_results = []
    for article in news_articles:
        title = article.get("title", "")
        summary = article.get("summary", "")
        timestamp = article.get("time_published", "")
        source = article.get("source", "")
        content = f"{title}. {summary}" if summary else title

        if content.strip():
            sentiment = sentiment_pipeline(content[:512])[0]  # Limiter à 512 caractères pour BERT
            sentiment_results.append({
                "title": title,
                "sentiment": sentiment["label"],
                "confidence": sentiment["score"],
                "published_at": timestamp,
                "source": source
            })

    return sentiment_results

# Affichage des résultats sous forme de DataFrame
def display_sentiment_results(results):
    df = pd.DataFrame(results)
    print("\nRésultats de l'analyse de sentiment :")
    display(df)

# === Script principal ===
def main():
    print("\nRécupération des news...")
    news_data = get_financial_news(ALPHA_VANTAGE_API_KEY, TICKER, LIMIT)

    if not news_data:
        print("Aucune news trouvée.")
        return

    print("\nAnalyse des sentiments...")
    sentiment_results = analyze_sentiment_with_finbert(news_data)

    if sentiment_results:
        display_sentiment_results(sentiment_results)
    else:
        print("Aucun résultat d'analyse de sentiment.")

if __name__ == "__main__":
    main()


Récupération des news...

Analyse des sentiments...


Device set to use cpu



Résultats de l'analyse de sentiment :


,title,sentiment,confidence,published_at,source
0,Apple Stock Analysis: Buy or Sell?,Positive,0.648492,20250201T104500,Motley Fool
1,3 Monster Stocks to Hold for the Next 10 Years,Positive,0.776459,20250201T104500,Motley Fool
2,Are These AI-Heavy Vanguard ETFs Still Good Pi...,Positive,0.992880,20250201T094700,Motley Fool
3,"Stocks Trim DeepSeek Losses, Fed Holds Steady,...",Positive,0.999809,20250131T213650,Benzinga
4,Apple Unusual Options Activity - Apple ( NASD...,Negative,0.996500,20250131T205446,Benzinga
5,EXCLUSIVE: This 'Magnificent 7' Stock Will Fee...,Neutral,0.999347,20250131T205446,Benzinga
6,Taiwan Bans DeepSeek AI Over Security Concerns...,Negative,0.989223,20250131T191242,Benzinga
7,Apple And Tesla Supplier STMicroelectronics Pl...,Negative,0.999912,20250131T174403,Benzinga
8,Apple's 'Better Than Feared' Guidance Calms iP...,Neutral,0.999925,20250131T173148,Benzinga
9,Wireless Headphones Market to Grow by USD 32.3...,Positive,0.999953,20250131T171900,Benzinga
